<center><h1>QUOTEMINER!! concept</h1></center>


Concept and testing notebook file for QuoteMiner. Cos jupyter variables really do come in handy cuh

In [3]:
import os
import whisperx
import subprocess
import pathlib
import torch
import json
from llama_index.core import  VectorStoreIndex, Settings, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from typing import List, Tuple
import dotenv
import unicodedata
import string
from collections import Counter
from difflib import SequenceMatcher




/Users/itzmaniss/coding/QuoteMiner/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/itzmaniss/coding/QuoteMiner/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


note: Should thread the youtube download script to not waste time. only network heavy


In [1]:
link = "https://www.youtube.com/watch?v=sYj0exUT_Mw&pp=ygUYYW5kcmV3IHRhdGUgdnMgdGhlcmFwaXN0"

In [10]:
subprocess.run([
    "yt-dlp",
    "-f",
    "bv*[ext=mp4]+ba",
    "--merge-output-format",
    "mp4",
    "-o",
    "./videos/%(title)s.%(ext)s",
    link,
    "--progress"
])# -"a", "your_links.txt"

[youtube] Extracting URL: https://www.youtube.com/watch?v=sYj0exUT_Mw&pp=ygUYYW5kcmV3IHRhdGUgdnMgdGhlcmFwaXN0
[youtube] sYj0exUT_Mw: Downloading webpage
[youtube] sYj0exUT_Mw: Downloading ios player API JSON
[youtube] sYj0exUT_Mw: Downloading mweb player API JSON
[youtube] sYj0exUT_Mw: Downloading m3u8 information
[info] sYj0exUT_Mw: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 1465
[download] Destination: ./videos/Andrew Tate vs Therapist (Round 2).f616.mp4
[download]  90.6% of ~   1.66GiB at    1.01MiB/s ETA 02:18 (frag 1326/1465)   [download] Got error: HTTPSConnectionPool(host='rr2---sn-npoldn76.googlevideo.com', port=443): Read timed out.. Retrying (1/10)...
[download] Got error: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')). Retrying (2/10)...
[download] 100% of    1.66GiB in 00:36:48 at 786.71KiB/s                      
[download] Destination: ./videos/Andrew Tate vs Therapist (Round 2).f251.w

CompletedProcess(args=['yt-dlp', '-f', 'bv*[ext=mp4]+ba', '--merge-output-format', 'mp4', '-o', './videos/%(title)s.%(ext)s', 'https://www.youtube.com/watch?v=sYj0exUT_Mw&pp=ygUYYW5kcmV3IHRhdGUgdnMgdGhlcmFwaXN0', '--progress'], returncode=0)

In [2]:
dotenv.load_dotenv()
#Defining constants

input_dir = pathlib.Path("./videos")
output_dir = pathlib.Path("./clips")
transcriptions_dir = pathlib.Path("./transcriptions")
hf_token = os.getenv("hugging_face_token")
#set if you have local whisperx model
model_path = None


#default pytorch variables
device = "cpu"
# batch_size = 
compute_type = "int8"
video_path = pathlib.Path("./short.mp4")

In [3]:
#Check for existing directories and make if needed

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(transcriptions_dir, exist_ok=True)

In [91]:
#Step 1: Transcription
def transcribe(video_path: pathlib.Path):
    model = whisperx.load_model("small", device=device, compute_type=compute_type)
    audio = whisperx.load_audio(str(video_path.absolute()))
    result = model.transcribe(audio, language="en", batch_size=8)
    complete_trancsript = ""
    for segment in result["segments"]:
        complete_trancsript += segment["text"]    
    model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
    refined_result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)
    word_timestamps = []
    # Iterate through each segment to extract word timestamps and confidence scores
    for segment in refined_result["segments"]:
        for word_info in segment.get("words", []):
            if "start" in word_info and "end" in word_info:
                word_timestamps.append({
                    "word": word_info["word"],
                    "start": word_info["start"],  # Assuming start and end are present
                    "end": word_info["end"],  # Assuming start and end are present
                    "score": word_info.get("score", 0)  # Default score if not provided
                })
    transript_path = transcriptions_dir / (video_path.stem + ".json")
    with open(transript_path, "w") as file:
        json.dump({"text": complete_trancsript, "word_timestamps": word_timestamps}, file)



In [ ]:
transcribe(pathlib.Path("/Users/itzmaniss/coding/QuoteMiner/videos/Dr. Marc Brackett： How to Increase Your Emotional Intelligence.mp4"))

In [80]:
from llama_index.core import Document

with open("./transcriptions/long.json", "r") as file:
        data = json.load(file)

x = Document(text=data["text"])

In [ ]:

# def extract_quotes(transcriptions_dir: pathlib.Path) -> list:
Settings.llm = Ollama(model="gemma2")
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

# documents = SimpleDirectoryReader(input_dir=transcriptions_dir).load_data()
index = VectorStoreIndex.from_documents([x,])

with open("prompt.txt", "r") as file:
    detailed_prompt = file.read()


query_engine = index.as_query_engine()
response = query_engine.query(detailed_prompt)

quotes = json.loads(remove_trailing_commas(y.strip().replace("```", "").strip().replace("json", "")))

with open("motivational_lines.txt", "w") as output_file:
    output_file.write(response.response)

    return quotes



In [ ]:

with open("prompt.txt", "r") as file:
    detailed_prompt = file.read()


query_engine = index.as_query_engine()
response = query_engine.query(detailed_prompt)

In [ ]:
import re

def remove_trailing_commas(json_string):
    """
    Remove trailing commas from a JSON string.

    Args:
        json_string (str): The JSON string to clean.

    Returns:
        str: The JSON string without trailing commas.
    """
    # Regex to detect trailing commas before a closing brace or bracket
    trailing_comma_pattern = re.compile(r",\s*([}\]])")

    # Replace all trailing commas with the closing brace/bracket
    cleaned_json = trailing_comma_pattern.sub(r"\1", json_string)

    return cleaned_json


In [6]:
def generate_clips(video_path: pathlib.Path, quotes: List[dict]) -> None:
    for idx, quote in enumerate(quotes):
        start = quote["start"]
        end = quote["end"]
        
        if start is None or end is None:
            print(f"Skipping quote {idx + 1} due to missing timestamps.")
            continue
        
        output_file = output_dir / f"{video_path.stem}_{idx + 1}.mp4"
        cmd = [
            "ffmpeg", "-i", video_path, "-ss", str(start), "-to", str(end),
            "-c", "copy", output_file
        ]
        subprocess.run(cmd)
    
